### **Importing Libraries**

In [5]:
!pip install optuna

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
from sklearn.cluster import KMeans

from sklearn.preprocessing import PolynomialFeatures

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')

### **Data Preprocessing**

In [7]:
train_df = pd.read_csv('Olympiad/train.csv')
X_df = train_df.drop(['PatientID','target'], axis = 1).copy()
y = train_df['target'].copy()

In [8]:
feat_interact = PolynomialFeatures(interaction_only = True, include_bias = False)
train_fi = feat_interact.fit_transform(X_df)
col_names = [f'col_{n}' for n in range(1,232)]
train_fi = pd.DataFrame(train_fi, columns = col_names)

In [9]:
imp_featues = ['col_208', 'col_34', 'col_22', 'col_57', 'col_53', 'col_26', 'col_119',
       'col_63', 'col_38', 'col_39', 'col_23', 'col_226', 'col_44', 'col_17',
       'col_74', 'col_42', 'col_37', 'col_6', 'col_75'] ## 'col_18',

X = pd.concat([X_df, train_fi[imp_featues]], axis = 1)

In [10]:
cluster_1 = ['HighBP', 'HighChol', 'CholCheck', 'BMI']
cluster_2 = ['Smoker', 'Stroke','Diabetes', 'PhysActivity']
cluster_3 = ['Fruits', 'Veggies', 'HvyAlcoholConsump','AnyHealthcare']
cluster_4 = ['NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth']

cluster_zip = [('cluster_2',cluster_2),('cluster_3',cluster_3),('cluster_4',cluster_4)]#('cluster_1',cluster_1),

### **Splitting Dataset**

In [11]:
strat_split = StratifiedShuffleSplit(n_splits=1, test_size = 0.25, random_state = 2020)
for train_index, test_index in strat_split.split(X, y):
    X_Train = X.iloc[train_index]; X_Test = X.iloc[test_index]
    y_Train = y.iloc[train_index]; y_Test = y.iloc[test_index]

In [ ]:
X_Train.shape, X_Test.shape

## **OPTUNA HYPERPARAMETER TUNING**

**Train-Test Split**

In [12]:
strat_split = StratifiedShuffleSplit(n_splits=1, test_size = 0.25, random_state = 2020)
for train_index, test_index in strat_split.split(X_Test, y_Test):
    X_train = X_Test.iloc[train_index]; X_test = X_Test.iloc[test_index]
    y_train = y_Test.iloc[train_index]; y_test = y_Test.iloc[test_index]

In [13]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1 - f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [14]:
def return_f1(params):
  model = XGBClassifier(**params, scale_pos_weight =9.7,  objective='binary:logistic')
  model.fit(X_train, y_train,  eval_set = [(X_test, y_test)], early_stopping_rounds = 200, verbose = 0, eval_metric = f1_eval)
  result = f1_score(y_test, model.predict(X_test))
  return result

In [ ]:
return_f1(params = {})

0.3673251468232781

### **Study 1**

In [15]:
def objective(trial):

  param = {
      'max_depth':trial.suggest_int('max_depth',3,10),
      'learning_rate':trial.suggest_loguniform('learning_rate',0.05,0.5),
      'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4,0.9),
      'subsample':trial.suggest_uniform('subsample',0.4,0.9),
      'reg_alpha': trial.suggest_uniform('reg_alpha',0,10),
      'random_state':trial.suggest_int('random_state',100,500),
      'nthread': -1
  }
  return(return_f1(param))

In [ ]:
study_1 = optuna.create_study(direction = 'maximize', sampler = TPESampler())
study_1.optimize(objective, n_trials = 600, show_progress_bar = True)

[I 2022-02-27 17:55:17,016] A new study created in memory with name: no-name-2729578f-ea76-4cc6-b0a6-4f6e4d681594


  0%|          | 0/600 [00:00<?, ?it/s]

[I 2022-02-27 17:55:23,820] Trial 0 finished with value: 0.3635928143712575 and parameters: {'max_depth': 6, 'learning_rate': 0.23172433051056457, 'colsample_bytree': 0.7791337049642622, 'subsample': 0.8659823197641556, 'reg_alpha': 0.20793908292924934, 'random_state': 210}. Best is trial 0 with value: 0.3635928143712575.
[I 2022-02-27 17:55:32,781] Trial 1 finished with value: 0.37644468904788103 and parameters: {'max_depth': 10, 'learning_rate': 0.20238405437049364, 'colsample_bytree': 0.5145927764555263, 'subsample': 0.5019470775329143, 'reg_alpha': 0.9734816322189299, 'random_state': 126}. Best is trial 1 with value: 0.37644468904788103.
[I 2022-02-27 17:55:38,403] Trial 2 finished with value: 0.3749679404975635 and parameters: {'max_depth': 6, 'learning_rate': 0.21677781212924319, 'colsample_bytree': 0.5975325488593272, 'subsample': 0.6902119797286022, 'reg_alpha': 5.43121370835048, 'random_state': 307}. Best is trial 1 with value: 0.37644468904788103.
[I 2022-02-27 17:55:44,507] 

### **Plotting of our search history**

In [ ]:
optuna.visualization.plot_optimization_history(study_1)

In [ ]:
optuna.visualization.plot_slice(study_1)

**Observation** The above plots suggest the folloeing parameter ranges

colsample_bytree = [0.7,0.9]

learning_rate = [0.3,0.5]

max_depth = [10]

random_state = [400,500]

reg_alpha = [0,1.5]

subsample = [0.45,0.65]